<a href="https://colab.research.google.com/github/sruthikoyya/SummerIntership_NITR/blob/main/UNet_NITR_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [ ]:
def normalize(datapoint):
    image = tf.image.resize(datapoint['image'], (572, 572))
    image = tf.cast(image, tf.float32) / 255.0

    mask = tf.image.resize(datapoint['segmentation_mask'], (388, 388))
    mask = tf.cast(mask, tf.float32)
    mask = tf.where(mask > 0, 1.0, 0.0)
    mask = tf.expand_dims(mask, axis=-1)

    return image, mask


In [ ]:
train_ds = tfds.load('oxford_iiit_pet:4.*.*', split='train[:80%]', as_supervised=False)
val_ds = tfds.load('oxford_iiit_pet:4.*.*', split='train[80%:]', as_supervised=False)

In [ ]:
train_ds = train_ds.map(normalize).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(normalize).batch(8).prefetch(tf.data.AUTOTUNE)

In [ ]:
#Convolution block
def conv_block(x, filters):
    x = tf.keras.layers.Conv2D(filters, 3, activation='relu', padding='valid')(x)
    x = tf.keras.layers.Conv2D(filters, 3, activation='relu', padding='valid')(x)
    return x

In [ ]:
#Crop and concat the contracting path to expansive path
def crop_and_concat(down, up):
    dh, dw = down.shape[1] - up.shape[1], down.shape[2] - up.shape[2]
    down = tf.keras.layers.Cropping2D(((dh//2, dh - dh//2), (dw//2, dw - dw//2)))(down)
    return tf.keras.layers.Concatenate()([down, up])

In [ ]:
def unet_model():
    inputs = tf.keras.Input(shape=(572, 572, 3))
    c1 = conv_block(inputs, 64)
    p1 = tf.keras.layers.MaxPool2D()(c1)

    c2 = conv_block(p1, 128)
    p2 = tf.keras.layers.MaxPool2D()(c2)

    c3 = conv_block(p2, 256)
    p3 = tf.keras.layers.MaxPool2D()(c3)

    c4 = conv_block(p3, 512)
    p4 = tf.keras.layers.MaxPool2D()(c4)

    c5 = conv_block(p4, 1024)

    u6 = tf.keras.layers.Conv2DTranspose(512, 2, strides=2, padding='valid')(c5)
    u6 = crop_and_concat(c4, u6)
    c6 = conv_block(u6, 512)

    u7 = tf.keras.layers.Conv2DTranspose(256, 2, strides=2, padding='valid')(c6)
    u7 = crop_and_concat(c3, u7)
    c7 = conv_block(u7, 256)

    u8 = tf.keras.layers.Conv2DTranspose(128, 2, strides=2, padding='valid')(c7)
    u8 = crop_and_concat(c2, u8)
    c8 = conv_block(u8, 128)

    u9 = tf.keras.layers.Conv2DTranspose(64, 2, strides=2, padding='valid')(c8)
    u9 = crop_and_concat(c1, u9)
    c9 = conv_block(u9, 64)

    outputs = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(c9)
    return tf.keras.Model(inputs, outputs)

In [ ]:
import tensorflow.keras.backend as K


def flatten(y):
    return K.flatten(tf.squeeze(y, axis=-1)) if len(y.shape) == 4 else K.flatten(y)

def precision_m(y_true, y_pred):
    y_true_f = flatten(y_true)
    y_pred_f = flatten(K.round(y_pred))

    tp = K.sum(y_true_f * y_pred_f)
    fp = K.sum((1 - y_true_f) * y_pred_f)
    return tp / (tp + fp + K.epsilon())

def recall_m(y_true, y_pred):
    y_true_f = flatten(y_true)
    y_pred_f = flatten(K.round(y_pred))

    tp = K.sum(y_true_f * y_pred_f)
    fn = K.sum(y_true_f * (1 - y_pred_f))
    return tp / (tp + fn + K.epsilon())

def f1_score(y_true, y_pred):
    prec = precision_m(y_true, y_pred)
    rec = recall_m(y_true, y_pred)
    return 2 * (prec * rec) / (prec + rec + K.epsilon())


In [ ]:
def iou(y_true, y_pred, smooth=1e-6):
    # Use tf.cond instead of Python if
    y_true = tf.cond(
        tf.equal(tf.rank(y_true), 5),
        lambda: tf.squeeze(y_true, axis=-1),
        lambda: y_true
    )
    y_pred = tf.cond(
        tf.equal(tf.rank(y_pred), 5),
        lambda: tf.squeeze(y_pred, axis=-1),
        lambda: y_pred
    )

    y_true_f = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred_f = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=1)
    total = tf.reduce_sum(y_true_f + y_pred_f, axis=1)
    union = total - intersection

    iou = tf.reduce_mean((intersection + smooth) / (union + smooth))
    return iou

In [ ]:

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.cond(
        tf.equal(tf.rank(y_true), 5),
        lambda: tf.squeeze(y_true, axis=-1),
        lambda: y_true
    )
    y_pred = tf.cond(
        tf.equal(tf.rank(y_pred), 5),
        lambda: tf.squeeze(y_pred, axis=-1),
        lambda: y_pred
    )

    y_true_f = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred_f = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=1)
    union = tf.reduce_sum(y_true_f, axis=1) + tf.reduce_sum(y_pred_f, axis=1)

    dice = tf.reduce_mean((2. * intersection + smooth) / (union + smooth))
    return dice

In [ ]:
model = unet_model()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[
                  'accuracy',
                  tf.keras.metrics.MeanIoU(num_classes=2),
                  dice_coef,
                  iou,
                  precision_m,
                  recall_m,
                  f1_score
              ])

In [ ]:
# Train
history = model.fit(train_ds,
                    epochs=10,
                    validation_data=val_ds)

Epoch 1/10
108/368 ━━━━━━━━━━━━━━━━━━━━ 12:00:25 166s/step - accuracy: 0.9972 - dice_coef: 0.9667 - f1_score: 0.9985 - iou: 0.9481 - loss: 0.0690 - mean_io_u: 0.4289 - precision_m: 1.0000 - recall_m: 0.9972

In [ ]:

#Evaluate on validation
results = model.evaluate(val_ds)
for name, val in zip(model.metrics_names, results):
    print(f"{name}: {val:.4f}")


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()
plt.show()
